# LangChain ReAct agent

In [ ]:
# ! pip3 install openai
# ! pip3 install langchain

In [ ]:
from agentquest.benchmarks.mastermind import MasterMindDriver, MasterMindUtils
from dotenv import load_dotenv

load_dotenv()
game = MasterMindUtils.load_data(data_path="__default__", category="4 digits")[4]
game

'8863'

Initialize the mastermind driver and get the first observation

In [2]:
driver = MasterMindDriver(game)
obs = driver.reset()  # Get the first observation
print(f"OBSERVATION: {obs.output}")

OBSERVATION: You are tasked to play the Mastermind game.
The host chooses a number and gives you the amount of digits. You have to guess the correct number as fast as you can.
The number can contain repetitions and any possible digit between: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9.
At each round, you provide a number as a guess. At each step, the host provides you this information:
1. The number of correct digits in the wrong position.
2. The number of correct digits in the correct position.
The game ends when the host outputs 'You Won!'
Carefully choose your strategy. Avoid brute force.
The guess must be in the following format:
Guess: <number>
Start guessing the 4 digits number.


Initialize the prompt and the ReAct agent

In [3]:
from langchain.agents import create_react_agent
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import StringPromptTemplate
from langchain.tools.base import BaseTool
from typing import List

prompt = (
    obs.output
    + """
Use the following tools:
{tools}

Use the following format:

Thought: summarize the info in the last observation and think about what to do.
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action.
Observation: the result of the action.
... (this Thought/Action/Action Input/Observation can repeat N times)
Final Answer: always provide the final answer or summarise what you did. Remember the keyword 'Final Answer' before

Input task: {input}

{agent_scratchpad}
"""
)

llm = AzureChatOpenAI(
    azure_deployment="gpt-4-default", api_version="2024-05-01-preview"
)


class CustomPromptTemplate(StringPromptTemplate):
    template: str
    tools: List[BaseTool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")

        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "

        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # print(thoughts)

        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join(
            [f"{tool.name}: {tool.description}" for tool in self.tools]
        )

        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])

        return self.template.format(**kwargs)


# Define a dummy tool for closed-box environments
class ProxyTool(BaseTool):
    name: str = "proxytool"
    description: str = "Provide the action you want to perform"

    def _run(self):
        pass

/tmp/ipykernel_277208/568656746.py:25: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(


In [4]:
prompt = CustomPromptTemplate(
    template=prompt,
    tools=[ProxyTool()],
    input_variables=[
        "input",
        "intermediate_steps",
        "tools",
        "tool_names",
        "agent_scratchpad",
    ],
)
agent = create_react_agent(llm, [ProxyTool()], prompt)
intermediate_steps = []

Start the Agent loop solving the task

In [5]:
step_cnt = 0
while not obs.success and obs.can_proceed and step_cnt < 5:
    # Retrieve the agent output
    agent_choice = agent.invoke(
        {"input": obs.output, "intermediate_steps": intermediate_steps}
    )

    print(agent_choice.tool_input)
    # Perform the step
    obs = driver.step_raw(agent_choice.tool_input)
    print(obs.output)
    # Update intermediate steps
    intermediate_steps.append((agent_choice, obs.output))

    step_cnt += 1

Guess: 1234
Wrong! Your guess has 0 correct digit in the correct position and 1 correct digit in the wrong position. Keep guessing.
Guess: 4321
Wrong! Your guess has 0 correct digit in the correct position and 1 correct digit in the wrong position. Keep guessing.
Guess: 2413
Wrong! Your guess has 1 correct digit in the correct position and 0 correct digit in the wrong position. Keep guessing.
Guess: 2513
Wrong! Your guess has 1 correct digit in the correct position and 0 correct digit in the wrong position. Keep guessing.
Guess: 2431
Wrong! Your guess has 0 correct digit in the correct position and 1 correct digit in the wrong position. Keep guessing.


In [7]:
driver.metrics.export(
    repetition_function_kwargs={"theta_a": 1, "num_execution_steps": 5}
)

{'goal': '8863',
 'success': False,
 'actions': [{'value': '1234'},
  {'value': '4321'},
  {'value': '2413'},
  {'value': '2513'},
  {'value': '2431'}],
 'states': [{'value': '1234'},
  {'value': '4321'},
  {'value': '2413'},
  {'value': '2513'},
  {'value': '2431'}],
 'observations': [{'output': 'Wrong! Your guess has 0 correct digit in the correct position and 1 correct digit in the wrong position. Keep guessing.',
   'success': False,
   'can_proceed': True},
  {'output': 'Wrong! Your guess has 0 correct digit in the correct position and 1 correct digit in the wrong position. Keep guessing.',
   'success': False,
   'can_proceed': True},
  {'output': 'Wrong! Your guess has 1 correct digit in the correct position and 0 correct digit in the wrong position. Keep guessing.',
   'success': False,
   'can_proceed': True},
  {'output': 'Wrong! Your guess has 1 correct digit in the correct position and 0 correct digit in the wrong position. Keep guessing.',
   'success': False,
   'can_proc